In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import uuid
import random
import re

### First, we will need to create a bank of customers : 

#### Let's fill our datasets with NaNs, for now. Let's say our sample contains 5000 customers.

In [2]:
uid_list = []
for i in range(5000):
    uid = str(uuid.uuid4())
    uid_list.append(uid)

#### Now, lets create the function that will simulate date of registrations of our hypotetical customers

In [3]:
def random_dates(start, end, n=5000):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2024-01-01')
date_list = random_dates(start, end)

date_list = pd.DatetimeIndex.tolist(date_list)

#### Time to create our cars

In [4]:
car_list = {'Hyundai':['Tucson', 'Accent', 'Elantra', 'Sonata', 'Santa_Fe'],
            'Honda':['Civic', 'CR-V', 'Accord', 'Pilot'],
            'Volkswagen':['Golf', 'Tiguan', 'Jetta'], 
            'BMW' : ['M4', 'M2', 'i5', 'X4', '3'],
            'Volvo' : ['XC60', 'XC40', 'S90', 'XC90'], 
            'Ford' :['Mustang', 'Focus', 'F-150', 'Transit', 'Expedition', 'Edge']}


car_make = []
car_model = []

for i in range(5000):
    a= list(car_list.keys())[(random.randint(0,5))]
    b= list(car_list[a])[random.randint(0, len(car_list[a])-1)]
    car_make.append(a)
    car_model.append(b)
    


In [5]:
year_list = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
car_year =[]

for i in range(5000):
    car_year.append(year_list[random.randint(0, len(year_list)-1)])

### Finally, we can create our customer's dataframe!

In [6]:
customers = pd.DataFrame({'customer_id' : uid_list, 'date_registered' : date_list, 'car_make' : car_make,
                          'car_model' : car_model, 'car_year' : car_year})


customers['date_registered'] = customers['date_registered'].dt.date

In [7]:
customers.head(10)

,customer_id,date_registered,car_make,car_model,car_year
0,5eb95ca6-4e6e-4a97-a7df-1c306fb77d4e,2021-06-09,Ford,Focus,2019
1,eb51e8eb-f4f5-4644-8080-847ff7c0d723,2023-02-18,Volvo,S90,2019
2,7f9f2599-cf72-4c1a-bc71-fe2c244e8751,2023-09-07,BMW,M2,2016
3,3092fed3-4d9a-403b-a5e2-9c53e42d18d0,2021-11-14,Honda,Pilot,2017
4,8869a21a-b1a4-415f-a9a7-4f3e0382bcc1,2021-07-08,BMW,X4,2022
5,6c2a261a-e505-43dc-ba86-7fc3b829c61c,2021-11-14,Hyundai,Sonata,2022
6,104eeafc-15a4-45c6-bce1-3061cd3d35d3,2021-01-22,BMW,3,2020
7,c2d439e9-43dd-4f8d-b416-928b90f85e35,2021-02-25,Volkswagen,Jetta,2017
8,71ceaab9-aa0d-4a6b-ba46-493f2e6b7c60,2021-01-07,BMW,M2,2020
9,ab079308-f7a4-45cb-a1bc-b8aed7ec74a9,2022-02-17,Hyundai,Santa_Fe,2016


### It's now time to create our transaction dataframe

#### Let's start by creating a random number of transactions for our clients.

#####  Let's start with dividing the customer by their registration date in group of six months. For example, a customer registered on the 1st of march 2021 is placed in the first six months group, a customer registered on the 5th of october 2021 in the six months to a year group and so on.

In [8]:
bins = [np.datetime64(date(2021,1,1)), np.datetime64(date(2021,7,1)), np.datetime64(date(2022,1,1)),
            np.datetime64(date(2022,7,1)), np.datetime64(date(2023,1,1)), np.datetime64(date(2023,7,1)),
            np.datetime64(date(2024,1,1))]
labels = ['0_6months','6_months', 'one_year', 'one_year_half', 'two_years', 'two_years_half']
customers['registration_range'] = pd.cut(customers['date_registered'], bins=bins, labels=labels, right=False)

 ##### I will then made a progressive selection through our customer dataframe selecting a random number between 5% and 35%, this represent the chance that a customer came for a service. 
 
 ##### The random function chooses a customer with replacement, meaning that the same customer may have come back during the same duration. I run the selection process starting with the first date group, then rerun it on the first and second, then first, second and third, and so on. That way, a customer that registered in the first six months have chances to return to the mechanic's shop more often than a customer that registered near the last 6 months.

In [9]:
customer_0_6months = customers[customers['registration_range'] == '0_6months']
customer_0_6months = list(customer_0_6months['customer_id'])
transactions0_6months = random.choices(customer_0_6months, k= int(len(customer_0_6months)* random.uniform(0.05,0.35)))


In [10]:
customer_6months = customers[(customers['registration_range'] == '0_6months') | (customers['registration_range'] == '6_months')]
customer_6months = list(customer_6months['customer_id'])
transactions6months = random.choices(customer_6months, k= int(len(customer_6months)*random.uniform(0.05,0.35)))

In [11]:
customer_one_year = customers[(customers['registration_range'] == '0_6months') | (customers['registration_range'] == '6_months')\
                             | (customers['registration_range'] == 'one_year')]
customer_one_year = list(customer_one_year['customer_id'])
transactions_one_year= random.choices(customer_one_year, k= int(len(customer_one_year)*random.uniform(0.05,0.35)))

In [12]:
customer_one_year_half = customers[(customers['registration_range'] == '0_6months') | (customers['registration_range'] == '6_months')\
                             | (customers['registration_range'] == 'one_year') | (customers['registration_range'] == 'one_year_half')]
customer_one_year_half = list(customer_one_year_half['customer_id'])
transactions_one_year_half= random.choices(customer_one_year_half, k= int(len(customer_one_year_half)*random.uniform(0.05,0.35)))

In [13]:
customer_two_years = customers[(customers['registration_range'] == '0_6months') | (customers['registration_range'] == '6_months')\
                             | (customers['registration_range'] == 'one_year') | (customers['registration_range'] == 'one_year_half')\
                              |  (customers['registration_range'] == 'two_years')]
customer_two_years = list(customer_two_years['customer_id'])
transactions_two_years= random.choices(customer_two_years, k= int(len(customer_two_years)*random.uniform(0.05,0.35)))

In [14]:
customer_two_years_half = customers[(customers['registration_range'] == '0_6months') | (customers['registration_range'] == '6_months')\
                             | (customers['registration_range'] == 'one_year') | (customers['registration_range'] == 'one_year_half')\
                              |  (customers['registration_range'] == 'two_years') | (customers['registration_range'] == 'two_years_half')]
customer_two_years_half = list(customer_two_years_half['customer_id'])
transactions_two_years_half= random.choices(customer_two_years_half, k= int(len(customer_two_years_half)*random.uniform(0.05,0.35)))

##### Our temporary dataframe "later_transactions" is all the transactions made by customer after their first. Let's set up a random date when those transactions occured, making sure that it's at least one day after their first transaction.

In [15]:
later_transactions = transactions0_6months + transactions6months + transactions_one_year + transactions_one_year_half \
                + transactions_two_years + transactions_two_years_half

In [16]:
later_transactions = pd.DataFrame({'customer_id' : later_transactions})

In [17]:
later_transactions = pd.merge(later_transactions,customers[['customer_id','date_registered']],on='customer_id', how='left')

In [18]:
#I'm reusing the same function as earlier, except that this time we remove the parameter "n" so we obtain only one random date

def random_dates(start, end):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u), unit='s')


end = pd.to_datetime('2024-01-02')

In [19]:
transaction_date = []

for index, row in later_transactions.iterrows() :
    start = pd.to_datetime(row['date_registered'] + timedelta(days=1))
    transaction_date.append(random_dates(start,end))
    
later_transactions.insert(loc=2, column='transaction_date', value=transaction_date)
later_transactions['transaction_date'] = later_transactions['transaction_date'].dt.date

#### Now let's create the column «transaction_date» for the transaction where our client registered

In [20]:
first_transaction = customers[['customer_id', 'date_registered']]
first_transaction.insert(loc=2, column='transaction_date', value= first_transaction['date_registered'])

#### Let's put those two dataframe we just created to have a full dataframe of all transactions that has been done

In [21]:
transactions = pd.concat([first_transaction, later_transactions], ignore_index = True)

#### Let's create a service ID for our reparations

In [22]:
transaction_id = []
for i in range(len(transactions)):
    uid = str(uuid.uuid4())
    transaction_id.append(uid)

In [23]:
transactions.insert(loc=3, column='transaction_id', value= transaction_id)

#### Now, let's generate some random services done on our clients' car

In [24]:
field_of_service = []
service = []
price = []

for i in range(len(transactions)):
    service_list = {'suspension' : {'shock_replace' : (random.randint(500,1200)), 'coil_spring' : (random.randint(550,700)),
                            'strut' : (random.randint(100,700)), 'control_arm' : (random.randint(200,300)),
                           'ball_joint' : (random.randint(250,350))},
           
           'brake' : {'pad_replacement' : (random.randint(115,250)), 'rotor_pad' : (random.randint(250,500)),
                      'full_replace' : (random.randint(400,900)), 'cleaning' : (random.randint(90,200))},
           
           'wheel' : {'install' : (random.randint(125,200)), 'buy_install' : (random.randint(600,1350)),
                      'balance' : (random.randint(90,125))},
           
           'ac_heating' : {'refrigerant_leak' : (random.randint(150,500)), 'fan' : (random.randint(650,800)),
                           'compressor' : (random.randint(400,600)), 'filters' : (random.randint(60,125))},
           
           'oil' : {'synth_change' : (random.randint(80,150)), 'reg_change' : (random.randint(60,110))},
           
           'engine' : {'rebuild' : (random.randint(2500,4000)), 'spark_plug' : (random.randint(150,400)),
                      'water_pump' : (random.randint(300,700)), 'timing_belt' : (random.randint(400,900))},
           
           'battery' : {'change' : (random.randint(100,300)), 'alternator' : (random.randint(300,800)),
                       }
          }
    a = list(service_list.keys())[(random.randint(0,len(service_list)-1))]
    b = list(service_list[a].keys())[random.randint(0, len(service_list[a])-1)]
    c = service_list[a][b]
    field_of_service.append(a)
    service.append(b)
    price.append(c)
    
transactions.insert(loc=4, column='field_of_service', value= field_of_service)
transactions.insert(loc=5, column='service', value= service)
transactions.insert(loc=6, column='price', value= price)
    

In [25]:
customers = customers.drop(columns = ['registration_range'])
transactions = transactions.drop(columns = ['date_registered'])

In [26]:
full_df = pd.merge(transactions, customers, on = 'customer_id', how = 'left')

#### Now that we have full datasets of customers and transactions, let's assing a 6 months group test to hypotetical customers who refered friends to our mechanic's shop. Let's say that 20% of customer refered someone

In [27]:
customers['recommended_someone'] = np.nan
customers['is_recommended'] = np.nan

test_group_list = full_df[(full_df['transaction_date']>= pd.to_datetime('2022-01-01').date()) &  \
                        (full_df['transaction_date'] <= pd.to_datetime('2022-07-01').date())]
test_group_list = test_group_list.drop_duplicates(subset=['customer_id'])

recommended = test_group_list.sample(n= int(len(test_group_list)*.2), replace=False)
recommended.loc[:, 'recommended_someone'] = True
test_group = recommended

In [28]:
customers = pd.merge(customers, test_group[['customer_id','recommended_someone']],on='customer_id', how='left')
customers = customers.drop(columns = 'recommended_someone_x')
customers = customers.rename(columns={"recommended_someone_y": "recommended_someone"})
customers = customers.fillna(False)
full_df = pd.merge(transactions, customers, on = 'customer_id', how = 'left')

#### Now, let's let's create a was_recommended column which matches our client who recommended_someone 

In [29]:
was_recommended_list = full_df[(full_df['transaction_date'] == full_df['date_registered']) & \
                               (full_df['date_registered']<= test_group[test_group['recommended_someone'] == True]['transaction_date'].max()+ timedelta(days=1)) &\
                              (full_df['date_registered'] > test_group[test_group['recommended_someone'] == True]['transaction_date'].min())\
                              &(full_df['recommended_someone'] == False)]
was_recommended_list = was_recommended_list.drop_duplicates(subset=['customer_id'])

In [30]:
was_recommended = was_recommended_list.sample(n= len(test_group[test_group['recommended_someone'] == True]),replace=False)
was_recommended.loc[:, 'is_recommended'] = True

In [31]:
customers = pd.merge(customers, was_recommended[['customer_id','is_recommended']],on='customer_id', how='left')
customers = customers.drop(columns = ['is_recommended_x'])
customers = customers.rename(columns={"is_recommended_y": "is_recommended"})
customers = customers.fillna(False)
customers = customers.drop_duplicates()
customers = customers.sort_values('date_registered')
full_df = customers.merge(transactions, on = 'customer_id', how = 'left')

##### We now need to give away the free oil change, both to customer who recommended and were recommended

In [32]:
free_oil_change = full_df[(full_df['recommended_someone'] == True) | (full_df['is_recommended'] == True)]


In [33]:
free_oil_cust_list = free_oil_change.drop_duplicates(subset=['customer_id'])
free_oil_cust_list = free_oil_cust_list['customer_id'].tolist()

##### People who already had an oil change 

In [34]:
free = full_df[(full_df['customer_id'].isin(free_oil_cust_list)) & (full_df['field_of_service'] == 'oil') & \
        (full_df['transaction_date'] > pd.to_datetime('2022-01-01').date())].drop_duplicates(subset=['customer_id'])

free_trans_list = free['transaction_id'].tolist()
free_cus_list = free['customer_id'].tolist()
transactions.loc[transactions.transaction_id.isin(free_trans_list), 'price'] = 0

In [35]:
transactions[transactions['price'] == 0]

,customer_id,transaction_date,transaction_id,field_of_service,service,price
30,be810749-2e92-495d-b703-a0c122a16d5a,2022-01-17,f36b97fd-c89a-47b7-b9b0-ff0fee402768,oil,reg_change,0
96,d865c04c-3e5a-488b-ab37-9e970fe357b0,2022-06-23,ca7aaa0b-2ece-4e7b-9f9e-41d319292666,oil,synth_change,0
282,a86f0c32-1e8d-4e9f-9493-427f4d87c166,2022-04-06,64d649c5-3a1e-42cc-a3bd-dfde6a4df8e2,oil,synth_change,0
288,4995b288-9dcc-4789-bdd5-42ac1ec37980,2022-06-24,d2cde6a4-3c90-4d80-aeaf-34a83b88d020,oil,reg_change,0
382,8d6bffa1-e483-417a-8908-e7587672c7f4,2022-01-21,14150982-ffe2-480d-b6a5-000dfb2f42ae,oil,reg_change,0
...,...,...,...,...,...,...
7916,76294d84-ce63-4ab5-86cb-e0c9c427292a,2023-10-22,9efd80de-3741-4fc5-9db5-2c5d3dca124f,oil,synth_change,0
7975,5693493d-8d39-4feb-855a-5c8b159cdefc,2023-05-04,2781401d-74fa-46ca-86f7-bad0ac69d97b,oil,synth_change,0
8056,dc835f99-9e16-42f2-9ebb-92139b4e79bb,2023-05-03,d8f84eea-85bd-4f0f-b74c-f151f10d53c3,oil,reg_change,0
8063,fcdbec87-cc7e-4294-bac3-964d8de047dd,2022-03-09,b206482e-65d3-44f6-bfc8-d160c1ed6d55,oil,synth_change,0


##### Create new oil change services FOR FREE

In [36]:
#Focusing on the customers who didn't have an oil change transaction
other_free = [x for x in free_oil_cust_list if x not in free_cus_list]

free_transaction = customers[customers['customer_id'].isin(other_free)]

In [38]:
#Creating transaction id

free_transaction_id = []
for i in range(len(free_transaction)):
    uid = str(uuid.uuid4())
    free_transaction_id.append(uid)
    
free_transaction.insert(loc=2, column='transaction_id', value= free_transaction_id)

In [39]:
#Creating a random transaction date

free_transaction_date = []

for index, row in free_transaction.iterrows() :
    start = pd.to_datetime(row['date_registered'] + timedelta(days=1))
    free_transaction_date.append(random_dates(start,end))

for i in range(len(free_transaction_date)):
    free_transaction_date[i] = free_transaction_date[i].date()


free_transaction.insert(loc=2, column='transaction_date', value=free_transaction_date)
free_transaction = free_transaction.drop(columns = ['date_registered', 'car_make', 'car_model', 'car_year', 'recommended_someone', 'is_recommended'])

In [40]:
#Randomizing synthetic or regular oil change

free_service = []

for i in range(len(free_transaction)):
    service_list = {'synth_change' : (random.randint(80,150)), 'reg_change' : (random.randint(60,110))}
    a = list(service_list.keys())[(random.randint(0,len(service_list)-1))]
    free_service.append(a)
    

free_transaction.insert(loc=3, column='field_of_service', value= 'oil')    
free_transaction.insert(loc=4, column='service', value= free_service)
free_transaction.insert(loc=5, column='price', value = 0)
    

In [41]:
#Adding new transactions to the list

transactions = pd.concat([transactions, free_transaction],ignore_index=True)
transactions = transactions.sort_values('transaction_date')

In [62]:
transactions.head(10)

,customer_id,transaction_date,transaction_id,field_of_service,service,price
4247,f23858f1-e9bf-4f33-9ddc-9b6a87a155ec,2021-01-01,f4d031d6-a99b-4938-b0e8-eecb29f07dab,brake,cleaning,140
4191,2e45afa9-e92e-43c1-9a88-2d7f8f815d8c,2021-01-01,56b9c0e7-e741-4baf-9865-b94068fa19b9,wheel,buy_install,1069
4786,7e2a2529-b83d-499c-9db8-680f9a45b9ba,2021-01-01,19d6a7fc-e907-40d2-adfe-159add0bf1d7,suspension,strut,110
1305,909ce425-ffbf-42c2-8b98-a81b16afe45f,2021-01-01,1b5c778c-4ec9-4a6d-a3fa-e25409d9131f,battery,alternator,570
829,8f44bfee-bf2c-4713-b47d-8b9910bd986e,2021-01-02,82e59169-b9be-4596-a42a-d7341b706e14,brake,pad_replacement,246
1005,3c587221-f7db-48eb-87ac-19bb23355aca,2021-01-02,b3c432f3-392b-4266-8343-27ca71374ac8,engine,rebuild,3495
3962,af7305cb-8d33-46cd-8c15-f9bb0672f1b5,2021-01-02,aa0222fb-bdb2-4473-ba36-cc7f338ef03d,wheel,buy_install,848
3038,cf6a322f-16d7-4b12-bad4-a0ad54ed8f9e,2021-01-02,4b8f733f-fb6f-4ab3-a291-aaa3a9a13550,oil,reg_change,81
1744,14712511-d5a3-48ca-83aa-dca747fcdfe2,2021-01-02,f60ceaba-d235-498a-ad59-07a35c200e74,wheel,balance,108
3833,d3e624d0-f3d7-44ec-92c6-06c59ef49bc9,2021-01-02,f9458092-61bd-49e2-937f-a905a2896b2a,ac_heating,compressor,591


In [50]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8538 entries, 4247 to 7150
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       8538 non-null   object
 1   transaction_date  8538 non-null   object
 2   transaction_id    8538 non-null   object
 3   field_of_service  8538 non-null   object
 4   service           8538 non-null   object
 5   price             8538 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 466.9+ KB


In [61]:
customers.head(10)

,customer_id,date_registered,car_make,car_model,car_year,recommended_someone,is_recommended
1305,909ce425-ffbf-42c2-8b98-a81b16afe45f,2021-01-01,Honda,Pilot,2019,False,False
4786,7e2a2529-b83d-499c-9db8-680f9a45b9ba,2021-01-01,Volvo,XC40,2023,False,False
4247,f23858f1-e9bf-4f33-9ddc-9b6a87a155ec,2021-01-01,Honda,Civic,2020,False,False
4191,2e45afa9-e92e-43c1-9a88-2d7f8f815d8c,2021-01-01,Volkswagen,Golf,2020,False,False
1744,14712511-d5a3-48ca-83aa-dca747fcdfe2,2021-01-02,Ford,Transit,2022,True,False
4697,a93444ca-d32e-4517-b15f-d3fb1db716bb,2021-01-02,Volkswagen,Golf,2016,False,False
1005,3c587221-f7db-48eb-87ac-19bb23355aca,2021-01-02,Volkswagen,Tiguan,2023,False,False
3456,5693493d-8d39-4feb-855a-5c8b159cdefc,2021-01-02,Ford,Edge,2020,True,False
3038,cf6a322f-16d7-4b12-bad4-a0ad54ed8f9e,2021-01-02,Hyundai,Accent,2020,False,False
3962,af7305cb-8d33-46cd-8c15-f9bb0672f1b5,2021-01-02,Ford,F-150,2017,False,False


In [44]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1305 to 4723
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   customer_id          5000 non-null   object
 1   date_registered      5000 non-null   object
 2   car_make             5000 non-null   object
 3   car_model            5000 non-null   object
 4   car_year             5000 non-null   object
 5   recommended_someone  5000 non-null   bool  
 6   is_recommended       5000 non-null   bool  
dtypes: bool(2), object(5)
memory usage: 244.1+ KB


In [47]:
full_df = customers.merge(transactions, on = 'customer_id', how = 'left')

In [60]:
full_df

,customer_id,date_registered,car_make,car_model,car_year,recommended_someone,is_recommended,transaction_date,transaction_id,field_of_service,service,price
0,909ce425-ffbf-42c2-8b98-a81b16afe45f,2021-01-01,Honda,Pilot,2019,False,False,2021-01-01,1b5c778c-4ec9-4a6d-a3fa-e25409d9131f,battery,alternator,570
1,909ce425-ffbf-42c2-8b98-a81b16afe45f,2021-01-01,Honda,Pilot,2019,False,False,2021-09-30,8b808d3b-144c-4fa8-89c3-b1349745fbef,suspension,strut,601
2,7e2a2529-b83d-499c-9db8-680f9a45b9ba,2021-01-01,Volvo,XC40,2023,False,False,2021-01-01,19d6a7fc-e907-40d2-adfe-159add0bf1d7,suspension,strut,110
3,7e2a2529-b83d-499c-9db8-680f9a45b9ba,2021-01-01,Volvo,XC40,2023,False,False,2022-02-09,d038d3fa-3e19-4661-ac51-bd581052e9e5,wheel,balance,120
4,f23858f1-e9bf-4f33-9ddc-9b6a87a155ec,2021-01-01,Honda,Civic,2020,False,False,2021-01-01,f4d031d6-a99b-4938-b0e8-eecb29f07dab,brake,cleaning,140
...,...,...,...,...,...,...,...,...,...,...,...,...
8533,0e3f6c20-4b45-49f1-8a47-9387d51009af,2023-12-30,BMW,M4,2016,False,False,2023-12-30,a9910042-8cab-4efc-aeb7-2a6ef4a6235b,suspension,strut,365
8534,9d763985-9edb-4803-837d-a60335d5d9cf,2023-12-30,BMW,i5,2016,False,False,2023-12-30,5924288d-958e-4fe8-b0f0-7b042b27996b,suspension,coil_spring,590
8535,0857325d-f994-4cc2-918d-90b5bc3087d6,2023-12-31,Honda,Pilot,2022,False,False,2023-12-31,09f07f36-bb36-432b-af05-fe07c01a16f8,brake,pad_replacement,146
8536,d8f0fe5b-c469-4f76-b0ba-ac168d5c27d8,2023-12-31,BMW,M2,2022,False,False,2023-12-31,30f3937f-08b6-42d8-8a07-278d3fc2da70,wheel,install,160
